In [1]:
import torch
from torch import nn
from torchvision import datasets
from torch.utils import data
from torchvision import transforms
from torchsummary import summary

In [2]:
class DenseBlock(nn.Module):
    def __init__(self, num, inchannel, outchannel):
        super().__init__()
        self.block = nn.Sequential()
        for i in range(num):
            self.block.add_module(f"convblock{i}", self.convblock(inchannel + i * outchannel, outchannel))

    def convblock(self, inchannel, outchannel):
        return nn.Sequential(
            nn.BatchNorm2d(inchannel),
            nn.ReLU(),
            nn.Conv2d(inchannel, outchannel, 3, padding=1),
        )

    def forward(self, x):
        for layers in self.block:
            y = layers(x)
            x = torch.cat((x ,y), dim=1)
        return x

In [3]:
temp = torch.rand(2, 3, 5, 5)
net = DenseBlock(2, 3, 10)
net(temp).shape

torch.Size([2, 23, 5, 5])

In [5]:
summary(net, (3, 5, 5), device="cpu")

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
       BatchNorm2d-1              [-1, 3, 5, 5]               6
              ReLU-2              [-1, 3, 5, 5]               0
            Conv2d-3             [-1, 10, 5, 5]             280
       BatchNorm2d-4             [-1, 13, 5, 5]              26
              ReLU-5             [-1, 13, 5, 5]               0
            Conv2d-6             [-1, 10, 5, 5]           1,180
Total params: 1,492
Trainable params: 1,492
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.01
Params size (MB): 0.01
Estimated Total Size (MB): 0.02
----------------------------------------------------------------
